# S3またはS3準拠ストレージから入力データや実験ソースコードを用意する
このノートブックでは、S3またはS3に準拠したストレージにあるデータを、この実験の入力データや実験ソースコードとして用意します。  

～ データガバナンス機能のデータ管理方法について ～  

データガバナンス機能では、大容量データの管理にあたって、データファイルのメタデータと実データとを分けて管理する技術「git-annex」を採用しています。「git-annnex」を活用して、大容量データの所在地を指すリンク情報を取得することで、データの実体とリンク情報を別に管理します。  
それによって、軽容量なリンク情報をデータガバナンス機能のリポジトリに保存するだけで、そのリポジトリのJupyter環境から実データにアクセス・ダウンロードすることができます。
  
以下はユースケースです。※後で画像をGithubにアップロードする。  
![image-2.png](attachment:image-2.png)

## 1. S3ストレージから入力データや実験ソースコードを用意する

### 1-A：S3にある単一データを用意する場合

以下のセルを実行して、リンクを作成したい場所のパスと、リンクを取得したいS3データのURLを入力してください。  
※入力に誤りがある場合は、再度このセルを実行して下さい。

In [ ]:
from ipywidgets import Text, Button, Layout
from IPython.display import clear_output
import datetime

style = {'description_width': 'initial'}

def on_click_callback(clicked_button: Button) -> None:
    global input_path
    global input_url
    input_path = text_path.value
    input_url = text_url.value
    clear_output()
    print("入力完了しました。")
    print("パス：" + input_path + "\nURL：" + input_url)

# テキストボックス
text_path = Text(
    description='リンクを作成したい場所のパス：',
    placeholder='/home/jovyan/experiments/ex-1/input_data/sample1',
    layout=Layout(width='600px'),
    style=style
)
text_url = Text(
    description='S3にあるファイルのURL：',
    placeholder='https://s3ds.sample.jp/sample/sample1',
    layout=Layout(width='600px'),
    style=style
)
button = Button(description='入力完了')
button.on_click(on_click_callback)
text_url.on_submit(on_click_callback)
display(text_path, text_url, button)

上のセルで入力いただいた内容でリンク情報を取得します。

In [ ]:
from colorama import Fore
try:
    result = !git annex addurl --fast --file=$input_path $input_url
    # 処理に失敗しても例外が発生しないため、以下の処理で失敗時に例外を発生させる
    for line in result:
        if 'failed' in line:
            raise Exception
except Exception:
    print(Fore.RED + 'リンク情報の取得に失敗しました。入力値を確認してください。\n')
else:
    print(Fore.BLACK + 'リンク情報の取得に成功しました。\n')
finally:
    for line in result:
        print(line)

リンク情報を元に実データを取得します。

In [ ]:
%cd ~/
try:
    result = !datalad get $input_path
    for line in result:
        if 'get(error)' or 'get(impossible)' in line:
            raise Exception
except Exception:
    print('データ取得に失敗しました。アクセス権限などを確認してください。')
else:
    print('データ取得に成功しました。2.の処理にお進みください。')

### 1-B：複数のデータを取得する場合

S3のバケットまたはバケット内のフォルダ単位で、データを取得します。  
AWSと接続するための情報を入力してください。

In [ ]:
import getpass
from IPython.display import clear_output
aws_access_key_id = getpass.getpass('AWS_ACCESS_KEY_ID：')
aws_secret_access_key = getpass.getpass('AWS_SECRET_ACCESS_KEY：')
aws_default_region = input('AWS_DEFAULT_REGION：')
!export AWS_ACCESS_KEY_ID=$aws_access_key_id
!export AWS_SECRET_ACCESS_KEY=$aws_secret_access_key
!export AWS_DEFAULT_REGION=$aws_default_region
clear_output()
print('入力を受け付けました。')

バケット名と、フォルダ単位で用意したい場合はフォルダパスを入力してください。

In [ ]:
bucket_name = input('S3バケット名：')
print('以下はバケットの任意のフォルダのデータを取得したい場合のみ入力してください。')
prefix = input('任意のフォルダパス：')
clear_output()
print('入力を受け付けました。')

S3のデータを、この実験の入力データとして取得するか、実験ソースコードを取得するかを選択してください。

In [ ]:
import json
from ipywidgets import Dropdown, Button, Layout
from IPython.display import clear_output

style = {'description_width': 'initial'}
choice1 = 'この実験の入力データとして扱う'
choice2 = 'この実験の実験ソースコードとして扱う'

# 実験を新規作成するか、実験パッケージをモニタリングするかを選択する
def on_click_callback(clicked_button: Button) -> None:
    global selected_choice
    if dropdown.value==choice1:
        selected_choice = 1
    elif dropdown.value==choice2:
        selected_choice = 2
    clear_output()
    print("入力を受けつけました：", dropdown.value)

dropdown = Dropdown(
    options=[choice1, choice2],
    description='リンク情報の保存先を選択してください：',
    disabled=False,
    layout=Layout(width='600px'),
    style=style
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
display(dropdown, button)

入力された内容でリンク情報の取得と実データの取得を行います。

In [ ]:
import boto3

#フォルダ内のファイル一覧を取得する
paths=[]
s3 = boto3.resource('s3') 
bucket = s3.Bucket(bucket_name)
if len(prefix)==0:
    response = bucket.meta.client.list_objects_v2(Bucket=bucket.name)
else:
    response = bucket.meta.client.list_objects_v2(Bucket=bucket.name, Prefix=prefix)
for content in response['Contents']:
    paths.append(content['Key'])
print(paths)

In [ ]:
import os
import glob
import csv

# データの用意先の実験パッケージパスを取得する
%store -r EXPERIMENT_TITLE
os.chdir('/home/jovyan')
package_path = glob.glob('**/'+EXPERIMENT_TITLE, recursive=True)[0]

# datalad addurlsのために、S3オブジェクトURLと情報リンク作成場所のリンクを用意する
objects=[]
for path in paths:
    url = "https://%s.s3.%s.amazonaws.com/%s" % (
        bucket_name,
        aws_default_region,
        path
    )
    if selected_choice==1:
        objects.append([package_path + '/input_data/S3/' + path, url])
    elif selected_choice==2:
        objects.append([package_path + '/source/S3/' + path, url])

%cd ~/
!mkdir .tmp
with open('.tmp/S3.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['who','link'])
    for obj in objects:
        writer.writerow([obj[0],obj[1]])

In [ ]:
# リンク情報を取得する
result = ''
try:
    result = !datalad addurls --fast .tmp/S3.csv '{link}' '{who}'
    for line in result:
        if 'addurl(error)' in line  or 'addurl(impossible)' in line:
            raise Exception
except Exception:
    print('リンク情報の取得に失敗しました。')
else:
    print('リンク情報の取得に成功しました。次の処理にお進みください。')
finally:
    print(result)

In [ ]:
# 実データを取得する
if selected_choice==1:
    path = package_path + '/input_data/S3'
elif selected_choice==2:
    path = package_path + '/source/S3'
try:
    result = !datalad get $path
    for line in result:
        if 'get(error)' in line or 'get(impossible)' in line:
            raise Exception
except Exception:
    print('データ取得に失敗しました。アクセス権限などを確認してください。')
else:
    print('データ取得に成功しました。次の処理にお進みください。')
finally:
    print(result)

## 2. リンク情報をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
import glob
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

%store -r EXPERIMENT_TITLE
os.chdir('/home/jovyan')
package_path = glob.glob('**/'+EXPERIMENT_TITLE, recursive=True)[0]
save_path = [ package_path + '/input_data']
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_入力データの準備', PATH = save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

In [ ]:
import os
import glob
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

save_path = [ package_path + '/source']
try:
    pm.execute_notebook(
        'WORKFLOWS/EX-WORKFLOWS/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = EXPERIMENT_TITLE + '_実験ソースコードの準備', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 3. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../experiment.ipynb)からトップページに遷移できます。  